In [7]:
# Install the necessary packages
!pip install accelerate transformers datasets evaluate torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
import torch
import os
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from huggingface_hub import login
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [16]:


login(token=os.getenv("HUGGINGFACE_TOKEN"))


In [13]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it", token=os.getenv("HUGGINGFACE_TOKEN"))


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2-2b-it.
401 Client Error. (Request ID: Root=1-67377a07-519e85a3528c8a6e51323e48;739ef217-06e1-4878-902b-62ad00ed38f4)

Cannot access gated repo for url https://huggingface.co/google/gemma-2-2b-it/resolve/main/config.json.
Access to model google/gemma-2-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
modelPT = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    token=os.getenv("HUGGINGFACE_TOKEN")
)

In [ ]:
# Sum the total number of parameters in the model
total_params = sum(p.numel() for p in modelPT.parameters())

# Display the total count of parameters
print(f"Total number of parameters in the modelPT: {total_params:,}")


In [ ]:
# Print the architecture of the modelPT
print(modelPT)


## Model Parameter Comparison: Reported vs Calculated

In this section, we compare the **reported parameters** for the Gemma 2B model (as per the technical paper) with the **calculated parameters** based on the loaded model `modelPT`.

### Reported Parameters for Gemma 2B Model:
According to the technical paper, the parameters for the **Gemma 2B model** are split into **Embedding Parameters** and **Non-embedding Parameters**:

- **Embedding Parameters**: 590,118,912
- **Non-embedding Parameters**: 2,024,517,888
- **Total Parameters**: 2,614,636,800

These values are the ones reported in the paper and give an overview of the architecture of the model. The **embedding parameters** come from the `embed_tokens` layer, and the **non-embedding parameters** account for the rest of the layers, including attention mechanisms and feedforward layers.

### Calculated Parameters for `modelPT`:
Upon loading the `Gemma 2B` model (`modelPT`), we performed calculations to estimate the parameters based on the architecture. Here’s how they break down:

1. **Embedding Parameters**:
   - From the `embed_tokens` layer, the calculation is:
     \[
     256000 \times 2304 = 590,118,912
     \]
   This matches the **reported embedding parameters**.

2. **Non-embedding Parameters**:
   - Based on the number of layers and components (such as attention, feedforward, and normalization layers), we calculate the **non-embedding parameters** as approximately:
     \[
     2,024,222,976
     \]
   This is very close to the reported **non-embedding parameters** of **2,024,517,888**.

3. **Total Parameters**:
   - The **total parameters** calculated from the `modelPT` are:
     \[
     590,118,912 \text{ (Embedding)} + 2,024,222,976 \text{ (Non-embedding)} = 2,614,341,888
     \]
   This is very close to the **reported total parameters** of **2,614,636,800**.

### Difference Between Reported and Calculated Parameters:
The **small difference** between the reported and calculated total parameters is approximately **295,000** parameters:
\[
2,614,636,800 - 2,614,341,888 = 295,000
\]
This discrepancy is minimal and could be due to rounding errors or slight variations in how the model was initialized or structured in the implementation.

### Conclusion:
- **Reported Total Parameters**: 2,614,636,800
- **Calculated Total Parameters**: 2,614,341,888

The calculated parameters for the `modelPT` are almost identical to the reported parameters, with a small difference of approximately 295,000. This is a negligible difference, indicating that the loaded model closely follows the architecture described in the paper.



# SST-2

In [9]:
# Load the SST-2 dataset
sst2 = load_dataset("glue", "sst2")

print(sst2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})


In [10]:
# Combine the train, validation, and test splits into a single dataset
combined_sst2 = concatenate_datasets([sst2['train'], sst2['validation'], sst2['test']])

# Print the structure of the combined dataset
print(combined_sst2)

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 70042
})


In [11]:
# Split the combined SST-2 dataset into 80% train and 20% test
train_test_split = combined_sst2.train_test_split(test_size=0.2, seed=1)

# Access the train and test splits
train_sst2 = train_test_split['train']
test_sst2 = train_test_split['test']

# Print the structure of both train and test splits
print(f"Train Split: {train_sst2}")
print(f"Test Split: {test_sst2}")


Train Split: Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 56033
})
Test Split: Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 14009
})


In [12]:
# Preprocess the data
def preprocess_data(examples):
    return tokenizer(examples["sentence"], truncation=True, padding=True)

# Apply preprocessing to both train and test datasets
train_sst2_tokenzd = train_sst2.map(preprocess_data, batched=True)
test_sst2_tokenzd= test_sst2.map(preprocess_data, batched=True)

# Print the structure of the preprocessed datasets
print(f"Preprocessed Train Split: {train_sst2_tokenzd}")
print(f"Preprocessed Test Split: {test_sst2_tokenzd}")


Map:   0%|          | 0/56033 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

In [23]:

# Assuming 'train_sst2_tokenzd' and 'test_sst2_tokenzd' are already tokenized datasets
# Custom dataset to wrap around Hugging Face datasets
class TextDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, idx):
        # Extract the tokenized data and label
        item = self.dataset[idx]
        return {
            'input_ids': torch.tensor(item['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(item['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(item['label'], dtype=torch.long)  # The target label
        }

    def __len__(self):
        return len(self.dataset)

# Create DataLoader instances
train_dataset = TextDataset(train_sst2_tokenzd)
test_dataset = TextDataset(test_sst2_tokenzd)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
import torch
import torch.nn as nn

# Define the custom model for sequence classification
class Gemma2ForSequenceClassification(nn.Module):
    def __init__(self, base_model, num_labels=2):
        super().__init__()
        self.base_model = base_model
        self.classifier = nn.Linear(base_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Get the outputs from the base model
        outputs = self.base_model(input_ids, attention_mask=attention_mask)
        # Extract the hidden state of the last token (classification token)
        hidden_state = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(hidden_state)
        return logits

# Load the pre-trained Gemma2 model
base_model = modelPT  # Assuming modelPT is already loaded

# Initialize the custom model
modelFT = Gemma2ForSequenceClassification(base_model)


In [ ]:


# Define the compute metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Set up training arguments
training_args_cls = TrainingArguments(
    output_dir="./results_sst2",                 # Directory to save the results
    evaluation_strategy="epoch",                 # Evaluate at the end of each epoch
    save_strategy="epoch",                       # Save model checkpoint after each epoch
    learning_rate=2e-5,                          # Learning rate
    per_device_train_batch_size=8,               # Batch size for training
    per_device_eval_batch_size=8,                # Batch size for evaluation
    num_train_epochs=3,                          # Number of epochs
    weight_decay=0.01,                           # Weight decay for regularization
)

# Initialize Trainer
trainer_cls = Trainer(
    model=modelFT,                                 # The model to fine-tune (your Gemma2ForSequenceClassification model)
    args=training_args_cls,                      # Training arguments defined above
    train_dataset=train_dataloader,                 # The preprocessed training dataset
    eval_dataset=test_dataloader,                   # The preprocessed test dataset
    compute_metrics=compute_metrics,             # Metrics function to compute during evaluation
)



In [ ]:
# # Fine-tune the model
# trainer_cls.train()



In [ ]:
# # Evaluate on the test set
# results_cls = trainer_cls.evaluate()
# print(results_cls)

# SQUAD

In [4]:
from datasets import load_dataset
# Load the SQuAD v2 dataset
squad_dataset = load_dataset("squad_v2")
print(squad_dataset)

ModuleNotFoundError: No module named 'datasets'

In [3]:
from datasets import concatenate_datasets
# Combine train and validation splits
combined_squad = concatenate_datasets([squad_dataset['train'], squad_dataset['validation']])

# Display the structure of the combined dataset
print(combined_squad)

ModuleNotFoundError: No module named 'datasets'

In [28]:
from datasets import DatasetDict

# Split the combined dataset into train and test (80:20 split)
combined_squad = combined_squad.train_test_split(test_size=0.2, seed=1)

# Create train and test splits
train_squad = combined_squad['train']
test_squad = combined_squad['test']

# Display the structure of the splits
print(f"Train Split: {train_squad}")
print(f"Test Split: {test_squad}")


Train Split: Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 113753
})
Test Split: Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 28439
})


In [35]:
example = train_squad[0]  # Accessing the first example in the train dataset
answers = example['answers']
print(answers['text'])  # Output: List of answers
print(answers['answer_start'])  # Output: List of start positions


['underground']
[32]


In [36]:
# Get the first few examples to inspect the answers field
def inspect_answers(dataset, num_samples=5):
    for i in range(num_samples):
        example = dataset[i]
        print(f"Example {i+1}:")
        print(f"Question: {example['question']}")
        print(f"Context: {example['context'][:500]}...")  # Print the first 500 characters of the context
        print(f"Answers: {example['answers']}")
        print("-" * 50)

# Call the function for the 'train_squad' dataset to get the first 5 examples' answers
inspect_answers(train_squad, num_samples=5)


Example 1:
Question: Where do many rodents live?
Context: Many rodents such as voles live underground. Marmots live almost exclusively above the tree line as high as 2,700 m (8,858 ft). They hibernate in large groups to provide warmth, and can be found in all areas of the Alps, in large colonies they build beneath the alpine pastures. Golden eagles and bearded vultures are the largest birds to be found in the Alps; they nest high on rocky ledges and can be found at altitudes of 2,400 m (7,874 ft). The most common bird is the alpine chough which can be ...
Answers: {'text': ['underground'], 'answer_start': [32]}
--------------------------------------------------
Example 2:
Question: How many people attend the Tulsa State Fair each year?
Context: During a 10-day run in Oklahoma City, the State Fair of Oklahoma attracts roughly one million people along with the annual Festival of the Arts. Large national pow-wows, various Latin and Asian heritage festivals, and cultural festivals such as 

In [1]:
def preprocess_qa(examples):
    questions = [q.strip() for q in examples["question"]]  # Ensure questions are stripped of extra spaces
    inputs = tokenizer(
        questions,
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Initialize lists to store start and end positions
    start_positions = []
    end_positions = []

    for i in range(len(examples["answers"])):
        answer_text = examples["answers"][i]["text"]  # Accessing the 'text' from each answer
        answer_start = examples["answers"][i]["answer_start"]  # Accessing the 'answer_start' from each answer

        # Find the end position of the answer
        end_position = answer_start + len(answer_text)  # Here, answer_start and answer_text are both scalar values, so this works
        start_positions.append(answer_start)
        end_positions.append(end_position)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs


# Tokenize the train and test datasets
train_squad_tokenized = train_squad.map(preprocess_qa, batched=True)
test_squad_tokenized = test_squad.map(preprocess_qa, batched=True)

# Print structure
print(f"Preprocessed Train Split: {train_squad_tokenized}")
print(f"Preprocessed Test Split: {test_squad_tokenized}")


NameError: name 'train_squad' is not defined

In [31]:
!pip install evaluate rouge_score nltk


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=059697446cbb87320ec0ab27592bc87177994e51e6a43cc8f56208b609a9af5c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [32]:
import evaluate
import nltk
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
from sklearn.metrics import f1_score

# Load the metrics
squad = evaluate.load("squad_v2")
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

# Function to compute the metrics
def compute_metrics(pred):
    # Get the true labels and predictions
    labels = pred.label_ids
    preds = pred.predictions

    # Decode the predictions and labels (the tokenizer will help with this)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute exact match
    exact_match = squad.compute(predictions=decoded_preds, references=decoded_labels)

    # Compute F1 score (using sklearn)
    f1 = f1_score(decoded_labels, decoded_preds, average='weighted')

    # Compute BLEU score
    bleu_score = bleu.compute(predictions=decoded_preds, references=decoded_labels)

    # Compute ROUGE score
    rouge_score = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    # Compute METEOR score
    meteor_score_value = meteor.compute(predictions=decoded_preds, references=decoded_labels)

    return {
        "exact_match": exact_match["exact_match"],
        "f1": f1,
        "bleu": bleu_score["bleu"],
        "rouge1": rouge_score["rouge1"],
        "rouge2": rouge_score["rouge2"],
        "rougeL": rouge_score["rougeL"],
        "meteor": meteor_score_value,
    }


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
import logging
from transformers import Trainer, TrainingArguments

# Set up logging to output to console
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Define the training arguments with logging at the end of each epoch
training_args_qa = TrainingArguments(
    output_dir="./results_squad",  # Directory where results will be saved
    evaluation_strategy="epoch",  # Evaluate at the end of every epoch
    save_strategy="epoch",        # Save checkpoints at the end of every epoch
    learning_rate=2e-5,           # Learning rate for fine-tuning
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,   # Batch size for evaluation
    num_train_epochs=3,             # Number of training epochs
    weight_decay=0.01,              # Weight decay to prevent overfitting
    logging_dir="./logs",           # Directory for logging
    logging_strategy="epoch",      # Log at the end of each epoch
)

# Define the Trainer
trainer_qa = Trainer(
    model=model_qa,
    args=training_args_qa,
    train_dataset=encoded_squad["train"],  # Train dataset
    eval_dataset=encoded_squad["test"],   # Test dataset
    tokenizer=tokenizer,                  # Use the tokenizer for encoding
    data_collator=default_data_collator,  # Handles padding for variable-length sequences
    compute_metrics=compute_metrics,
)



In [ ]:
# Start training with logging at the end of each epoch
trainer_qa.train()


In [ ]:
# Evaluate the model after training
results_qa = trainer_qa.evaluate()

# Print the evaluation results
print(results_qa)